In [1]:
import matplotlib.pyplot as plt
import ultralytics
import pandas as pd

import numpy as np
import cv2
import os

from skimage import io, color, img_as_ubyte
from skimage.filters import gaussian, threshold_otsu
from skimage.exposure import equalize_adapthist

from PIL import Image
from utils import display_images_in_rows, prepare_labels, plot_image_comparison_with_zoom

# Definir número de GPUs a utilizar
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['MEMORY_GROWTH'] = '1'
os.environ['PYTHONWARNINGS'] = 'ignore'

import ImageEnhancer as IE
import ImageTextExtractor as ITE
import ImagePromptResponseGenerator as IPRG


pd.options.mode.chained_assignment = None

[2023-12-07 03:18:21,729] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# Extracción y Análisis Automatizado de Características en Caricaturas del Siglo XIX Latinoamericano: Una Integración de Procesamiento de Imágenes y Machine Learning

## Autores

* David Santiago Ortiz Almanza
* Juan Sebastián Pinzón Roncancio

## Resumen (Abstract)

Las caricaturas, son una fuente rica en contexto histórico y cultural, ofrecen una perspectiva única sobre las dinámicas sociopolíticas de su época. Sin embargo, su interpretación se ve limitada por la complejidad de su análisis manual y la degradación física de los documentos. El objetivo de este proyecto es automatizar y mejorar la eficiencia de dicho proceso, facilitando el análisis e interpretación de estas valiosas fuentes.

En términos de los métodos utilizados, se adoptó un enfoque híbrido que combina técnicas de procesamiento de imágenes tradicionales con técnicas avanzadas de machine learning. El proceso de preprocesamiento implicó la digitalización de alta calidad y la aplicación de algoritmos para mejorar la claridad y legibilidad de las imágenes. Para la extracción de textos, se optó por utilizar reconocimiento óptico de caracteres (OCR), usando como arquitectura redes convolucionales recurrentes. Por su parte, la identificación y etiquetación de elementos gráficos, se realizó utilizando YOLO v8, un modelo de detección de objetos de vanguardia. Finalmente, se utilizó LLaVA 1.5 como modelo largo multimodal (LMM) para la generación automatizada de descripciones.

Los resultados obtenidos fueron notables: se logró mejorar la calidad de las imágenes, extraer los textos presentes con un índice de Jaccard de 0.47 y generar descripciones coherentes y contextualizadas con un F1 BERT Score de 0.25, demostrando un buen desempeño. Sin embargo, se presentaron desafíos en la identificación de figuras dentro de las caricaturas, evidenciado por un índice de Jaccard de 0.13 en esta tarea. Asimismo, el sistema demostró un desempeño eficiente, lo que permitió el procesamiento de un gran volumen de imágenes en poco tiempo. Este proyecto no solo representa un avance significativo en la preservación digital y el análisis del patrimonio cultural, sino que también establece un nuevo estándar en la interpretación automatizada de fuentes visuales históricas.

## Introducción

En el estudio de la historia y la cultura latinoamericana, las caricaturas del siglo XIX representan una fuente visual invaluable. Estas obras no solo reflejan las tendencias artísticas de su tiempo, sino que también actúan como crónicas visuales de los acontecimientos sociopolíticos y las actitudes culturales. Sin embargo, el análisis de estas caricaturas es inherentemente complejo y requiere de una considerable inversión de tiempo y conocimiento especializado. A menudo, estos valiosos documentos históricos sufren de degradación, lo que dificulta aún más su interpretación y análisis.

Reconociendo la importancia de preservar y facilitar el acceso a este patrimonio cultural, nuestro proyecto se enfoca en la aplicación de tecnologías avanzadas de procesamiento de imágenes y aprendizaje automático. El objetivo es doble: primero, mejorar la legibilidad y la calidad de las imágenes de las caricaturas; y segundo, automatizar y optimizar el proceso de análisis e interpretación. Este enfoque no solo ahorra tiempo y recursos, sino que también abre nuevas posibilidades para la investigación y el estudio de estas fuentes.

Las contribuciones de este trabajo son significativas. Mediante la combinación de técnicas de procesamiento de imágenes, tanto tradicionales como avanzadas, y el uso de reconocimiento óptico de caracteres, así como la implementación de modelos de detección de objetos como YOLO v8 y modelos largos multimodales como LLaVA 1.5, hemos logrado automatizar la extracción de texto, la identificación de elementos gráficos y la generación de descripciones integradas. Este enfoque innovador mejora la eficiencia y facilita el análisis de las caricaturas latinoamericanas del siglo XIX.

Este trabajo no solo contribuye al campo de la digitalización y análisis de documentos históricos, sino que también establece un precedente en la aplicación de tecnologías avanzadas para el estudio de la cultura visual latinoamericana del siglo XIX.

El presente documento se estructura de la siguiente manera:

* **Estado del Arte** - Presentamos los trabajos previos en el área y resaltamos las limitaciones de los mismos.
* **Métodos** - Detallamos las técnicas y herramientas específicas empleadas en el proyecto.
* **Resultados** - Mostramos los hallazgos y logros obtenidos a través de nuestro enfoque.
* **Discusión y Conclusión** - Discutimos las implicaciones de nuestros resultados y concluimos con reflexiones finales.
* **Bibliografía** - Proporcionamos referencias de todas las fuentes utilizadas.

## Estado del Arte


Para ser precisos, el procesamiento y análisis de imágenes de imprenta del siglo XIX no ha sido explorado en profundidad, o al menos no se conocen publicaciones significativas sobre el tema. Es crucial destacar que, hasta la fecha, no existe un conjunto de datos extenso que incluya imágenes caricaturescas de la imprenta latinoamericana del siglo pasado, acompañadas del contenido etiquetado relevante que este proyecto busca analizar. En el caso de disponer de un conjunto de datos amplio, sería de interes considerar etiquetas como el texto contenido en la caricatura, los elementos detectados en la imagen, la presencia de entidades antropomórficas, y una descripción textual de la imagen, entre otros aspectos. Es importante mencionar que Laura Manrique, estudiante de doctorado en Historia de la Universidad de Los Andes, está actualmente trabajando en la construcción de dicho conjunto de datos.

Aún así, es valioso considerar por separado cuál es el estado del arte de cada uno de los submodulos propuestos frente a las imagenes historicas: 

- **Preprocesamiento de imágenes [4]**

    En la esfera del preprocesamiento de imágenes aplicado a documentos históricos, el estudio de Hong Xia Wang, Bang Song, Jian Chen y Yi Yang, titulado "Degraded Document Image Preprocessing Using Local Adaptive Sharpening and Illumination Compensation" (2022), representa un avance significativo. Este trabajo se enfoca en la restauración de imágenes de documentos que han experimentado una considerable degradación a lo largo del tiempo. Los autores introducen un enfoque novedoso que combina técnicas de agudización adaptativa local con métodos de compensación de iluminación. Estas técnicas se emplean para realzar detalles sutiles y corregir irregularidades en la iluminación, mejorando así la legibilidad y calidad visual de documentos deteriorados. 
    

- **OCR (Optical Character Recognition) [5]**

    En el estudio "Text Extraction and Restoration of Old Handwritten Documents" por Mayank Wadhwani y colaboradores, se aborda la desafiante tarea de extraer textos de documentos históricos manuscritos que han sufrido severas degradaciones a lo largo del tiempo. Los autores presentan dos innovadoras metodologías basadas en redes neuronales profundas, específicamente diseñadas para abordar los desafíos únicos presentes en la extracción de texto de documentos antiguos. 

- **Detección de imagenes**

    Uno de los modelos estado del arte para detección de objetos es YOLOv8 (You Only Look Once), no obstante, no existe a la fecha ningun modelo YOLO pre-entrenado que consiga resolver la tarea de detección y clasificación para imagenes caricaturezcas de imprenta del siglo XIX. 

- **Descriptor de imágenes en base a LMM's** 

    El paper "Visual Instruction Tuning" explora el afinamiento instruccional de grandes modelos de lenguaje (LLMs) en el ámbito multimodal, introduciendo LLaVA, un modelo multimodal grande entrenado de extremo a extremo que integra un codificador visual y un LLM para entender visual y lingüísticamente de manera general. Este enfoque puede ser relevante para el ajuste e implementación de los modelos de Large Multinomial Models, ya que sugiere una metodología para conectar interpretaciones visuales y de lenguaje, que podría ser adaptada para generar descripciones automáticas que contextualicen la interacción entre una imagen y el texto asociado [2].

## Método

El proyecto emplea una arquitectura modular interconectada para el análisis automatizado de caricaturas históricas. Esta arquitectura esta conformada por cuatro componentes principales: preprocesamiento, detección de objetos, extracción de textos, y generación de descripciones.

La arquitectura de la solución propuesta inicia con una caricatura histórica en su forma original. Esta imagen entra en un módulo de preprocesamiento, donde se aplican algoritmos de mejora de imagen para realzar la calidad visual. Posteriormente, esta imagen mejorada se introduce simultáneamente en dos módulos: el de extracción de textos y el de detección de objetos. El primero se encarga de identificar cualquier texto presente, mientras que el segundo reconoce y clasifica los elementos gráficos de la caricatura. Estos procesos generan una lista de las palabras y objetos detectados. En la etapa final los datos generados anteriormente —la imagen preprocesada, los textos extraídos y los objetos identificados— convergen para formar la entrada de un modelo de lenguaje multimodal, el cual se encarga de generar una descripción detallada de la caricatura.

La ***Figura 1*** ilustra esta arquitectura propuesta, delineando claramente el flujo de procesamiento desde la imagen original hasta la obtención de una descripción de la misma.

<figure>
<center><img src="Paper Images/Arquitectura General de la Solución.png" style="width:80%"></center>
<figcaption align = "center"> Figura 1. Arquitectura General de la Solución</figcaption>
</figure>

A continuación, se presenta una descripción detallada del método utilizado para cada uno de estos módulos.

#### **Preprocesamiento**

- **Algoritmo de recorte Inteligente**

Al observar las imágenes proporcionadas, se puede observar que la gran mayoría de las imágenes tienen un exceso de área vacía que rodea las ilustraciones centrales. Estos espacios blancos no aportan información y pueden quitarle foco a la ilustración principal. Es por esta razón que se diseñó un algoritmo que se denominó "Algoritmo de recorte Inteligente", el cual elimina áreas de fondo vacío o insignificante alrededor del objeto principal de la imagen. 

A continuación se explica de manera detallada el algoritmo implementado:

1. **Conversión a escala de grises:** La imagen se convierte a una representación en escala de grises, lo que simplifica su procesamiento posterior.

2. **Umbralización:** A la imagen en escala de grises se le aplica un umbral, donde los píxeles con valores mayores que 220 se consideran como parte del objeto principal y se establecen en True (verdadero), mientras que los píxeles con valores menores o iguales a 220 se consideran como fondo y se establecen en False (falso).

3. **Búsqueda del cuadro delimitador horizontal:** Se suman los valores de todos los píxeles en cada fila de la imagen umbralizada. Luego, se identifica el primer y último índice $(y_0, y_1)$ de la fila donde la suma es inferior al 90% del ancho de la imagen, lo que indica el inicio y el final del objeto principal en la dirección vertical.

4. **Búsqueda del cuadro delimitador vertical:** De manera similar, se suman los valores de todos los píxeles en cada columna de la imagen umbralizada. Luego, se el primer y último índice $(x_0, x_1)$ de la columna donde la suma es diferente de cero, lo que indica el inicio y el final del objeto principal en la dirección horizontal.

5. **Recorte de la imagen:** Finalmente, se utiliza la información obtenida en los pasos anteriores para recortar la imagen original de manera que solo quede el objeto principal, eliminando cualquier espacio en blanco o insignificante alrededor del objeto.

La **Figura 2** ilustra el algoritmo ilustra gráficamente el macroalgoritmo del recorte inteligente implementado.


<figure>
<center><img src="Paper Images/Macroalgoritmo Recorte Inteligente.png" style="width:80%"></center>
<figcaption align = "center"> Figura 2. Macroalgoritmo Recorte Inteligente</figcaption>
</figure>

A continuación, se presenta la implementación del algoritmo de recorte inteligente en Python, haciendo uso de Numpy y PIL.

In [ ]:
def recorte_inteligente(image_path):
    # Open the image
    img = Image.open(image_path)

    # Convert the image to grayscale
    gray_img = img.convert("L")

    # Convert the grayscale image to numpy array and threshold
    img_array = np.array(gray_img)
    thresholded = (img_array > 220)

    # Find the horizontal bounding box
    horizontal_sum = np.sum(thresholded, axis=1)
    y0 = np.where(horizontal_sum < thresholded.shape[1] * 0.9)[0][0]
    y1 = np.where(horizontal_sum < thresholded.shape[1] * 0.9)[0][-1]

    # Find the vertical bounding box
    vertical_sum = np.sum(thresholded, axis=0)
    x0 = np.where(vertical_sum < thresholded.shape[0])[0][0]
    x1 = np.where(vertical_sum < thresholded.shape[0])[0][-1]

    # Crop the image to the bounding box
    cropped_img = img.crop((x0, y0, x1, y1))
    
    return cropped_img

Seguidamente, se aplica el algoritmo de recorte inteligente a todas las imágenes originales almacenadas en la carpeta "original_images" y almacena el resultado en la carpeta "cropped_images".

In [ ]:
# Apply crop algorithm to all images

crop_input_path = 'original_images'
crop_output_path = 'cropped_images'

if not os.path.exists(crop_output_path):
    os.makedirs(crop_output_path)

for i in range(1, 102):
    cropped_img = recorte_inteligente(f"{crop_input_path}/{str(i)}.jpg")
    cropped_img_array = np.array(cropped_img)
    cropped_img_array = cv2.cvtColor(cropped_img_array, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f"{crop_output_path}/{str(i)}.jpg", cropped_img_array)

- **Mejora de la Calidad Visual Mediante Técnicas Tradicionales de Procesamiento de Imágenes**

La inspección detallada del conjunto de imágenes reveló una considerable presencia de ruido visual, originado por la calidad del papel, las técnicas de impresión utilizadas y la degradación a lo largo del tiempo. En particular, se identificaron ruidos de textura y patrón, así como manchas y arrugas. Estas irregularidades comprometen significativamente la calidad y uniformidad de las imágenes, presentando desafíos para su análisis en etapas subsiguientes. Para abordar estos problemas, se diseñó una secuencia de preprocesamiento que incorpora técnicas tradicionales de procesamiento de imágenes, las cuales han demostrado mejorar notablemente la calidad visual de las imágenes.

El algoritmo implementado se describe a continuación:

1. **Conversión a Escala de Grises:** La primera acción es convertir la imagen a escala de grises, lo que simplifica el procesamiento al reducir la complejidad del color, focalizando la atención en la intensidad de la luz.

2. **Aplicación del Filtro Gaussiano:** Posteriormente, se aplica un filtro Gaussiano para suavizar la imagen. Este paso atenúa el ruido y homogeneiza la textura sin comprometer los detalles esenciales.

3. **Mejora de Contraste con CLAHE:** El método CLAHE (Contrast Limited Adaptive Histogram Equalization) se utiliza sobre la imagen suavizada para realzar el contraste. Esta técnica de ecualización del histograma mejora la definición y resalta los detalles sutiles, preservando la estructura global de la imagen.

4. **Binarización con Umbral Adaptativo:** La imagen es finalmente binarizada utilizando el método de Otsu, que establece automáticamente el umbral óptimo. Para refinar este proceso, se multiplica el umbral determinado por Otsu por un factor de 0.65, un valor obtenido mediante pruebas visuales iterativas que mejoran la precisión de la binarización.

La **Figura 3** presenta una representación esquemática de este algoritmo de preprocesamiento, ilustrando de forma clara los pasos secuenciales aplicados para la mejora tradicional de las imágenes.

<figure>
<center><img src="Paper Images/Macroalgoritmo Preprocesamiento Tradicional.png" style="width:80%"></center>
<figcaption align = "center"> Figura 3. Preprocesamiento mediante Técnicas Tradicionales</figcaption>
</figure>

A continuación, se presenta la implementación del algoritmo de preprocesamiento de imágenes descrito anteriormente implementado en Python, haciendo uso de la librería Scikit-Image.

In [9]:
def preprocess_image(image_path, binarize=False):
    # Cargar la imagen
    image = io.imread(image_path)

    # Convertir a escala de grises
    gray_image = color.rgb2gray(image)

    # Aplicar filtro Gaussiano
    blurred_image = gaussian(gray_image, sigma=0.5)

    # Aplicar CLAHE
    clahe_image = equalize_adapthist(blurred_image)

    # Binarización opcional
    if binarize:
        thresh = threshold_otsu(clahe_image)
        binary_image = clahe_image > thresh * 0.65
        return image, gray_image, blurred_image, clahe_image, binary_image

    return image, gray_image, blurred_image, clahe_image

Ahora, se aplica el algoritmo de preprocesamiento a través de técnicas tradicionales a todas las imágenes almacenadas en la carpeta "cropped_images" y se almacena el resultado en la carpeta "cropped_traditional_processing_images".

In [10]:
traditional_processing_input_path = 'cropped_images'
traditional_processing_output_path = 'cropped_processed_images'

if not os.path.exists(traditional_processing_output_path):
    os.makedirs(traditional_processing_output_path)
    
for i in range(1, 102):
    processed_image = img_as_ubyte(preprocess_image(f"{traditional_processing_input_path}/{str(i)}.jpg", True)[-1])
    io.imsave(f"{traditional_processing_output_path}/{str(i)}.jpg", processed_image)

- **Realce de la resolución y calidad con Real-ESRGAN**

La técnica Real-ESRGAN (Enhanced Super-Resolution Generative Adversarial Networks) consiste en una metodología avanzada de inteligencia artificial diseñada para mejorar la calidad de imágenes a través del proceso de super-resolución. Utiliza una arquitectura de redes generativas adversarias (GANs) que comprende dos componentes clave: un generador y un discriminador. El generador se encarga de transformar imágenes de baja resolución en versiones de alta resolución, mientras que el discriminador evalúa la calidad de estas imágenes mejoradas, ayudando al generador a perfeccionar su capacidad para producir imágenes realistas y de alta calidad. Este proceso se lleva a cabo mediante ciclos iterativos de retroalimentación entre ambos componentes, con el objetivo de alcanzar un resultado donde la imagen mejorada es casi indistinguible de una imagen de alta resolución real.

En el caso específico de imágenes caricaturescas, Real-ESRGAN ofrece un modelo preentrenado denominado REALSR-6B_ANIME. Este modelo, que cuenta con aproximadamente 6 mil millones de parámetros, ha sido entrenado exclusivamente con imágenes de animación y caricaturas. Esto significa que está específicamente optimizado para manejar las características únicas de este tipo de imágenes, como estilos artísticos distintivos, líneas claras y colores vibrantes. Su especialización en animación y caricaturas lo hace ideal para mejorar la resolución y la calidad de imágenes con estas características, asegurando que los detalles y el estilo artístico se mantengan fieles al original mientras se incrementa su resolución, ideal para restituir la resolución caricaturas antiguas.

In [ ]:
ESRGAN = IE.ImageEnhancer(input_path='images_prepared_for_enhancement', 
                        model_name='RealESRGAN_x4plus_anime_6B', 
                        output_path='results_enhancement_real',
                        denoise_strength=0.5, 
                        outscale=2, 
                        suffix='', 
                        face_enhance=False, 
                        fp32=False, 
                        gpu_id=1)

In [ ]:
ESRGAN.enhance_images()

#### **Extracción de Textos (OCR)**

Para la extracción de texto de las caricaturas, se utilizó el paquete Keras-OCR desarrollado por Fausto Morales [1], que integra el modelo de detección de texto CRAFT (Character-Region Awareness For Text detection) [2] y una implementación en Keras de CRNN (Convolutional Recurrent Neural Networks) [3] para el reconocimiento de texto.

Además, se siguió una metodología propuesta por Ellie Kuang [4] para organizar el texto extraído en un formato coherente con la lectura humana en español, de izquierda a derecha y de arriba a abajo. Este paso es crucial para la utilidad del texto en la generación subsiguiente de descripciones automáticas.

Se desarrolló un módulo en Python muy sencillo de utilizar que combina todas estas funcionalidades, diseñado para procesar imágenes en lotes de un tamaño definido para adaptarse a GPUs con diferentes capacidades de memoria. Este módulo también permite la inclusión de un archivo con los textos reales para evaluar el rendimiento utilizando el índice de Jaccard.

A continuación, se realiza la inicialización de dicho módulo y se procesan todas las imágenes almacenadas en la carpeta "cropped_images" y las imágenes anotadas se almacenan en la carpeta "cropped_OCR_images".

In [2]:
OCR_origin_folder = "cropped_images"
OCR_destination_folder = "cropped_processed_OCR_images"

if not os.path.exists(OCR_destination_folder):
    os.makedirs(OCR_destination_folder)

OCR = ITE.ImageTextExtractor(images_paths=[OCR_origin_folder+"/"+path for path in os.listdir(OCR_origin_folder) if ".jpg" in path],
                             save_annotated_images=True,
                             save_folder=OCR_destination_folder,
                             batch_size=5)

OCR.process_all_images()

Looking for /home/jspinzonr/.keras-ocr/craft_mlt_25k.h5
Looking for /home/jspinzonr/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 0s 221ms/step
Todas las imágenes procesadas con éxito.


#### **Detección - Modelo de Detección con YOLOv8**



#### **Generación de Descripciones Detalladas Utilizando LMM**

El último paso en nuestra arquitectura de solución implica la generación de una descripción detallada de la caricatura, aprovechando las salidas procesadas de los módulos anteriores: la imagen mejorada, así como los textos y objetos identificados. Para llevar a cabo esta tarea, empleamos un modelo de lenguaje multimodal de gran tamaño (LMM), específicamente LLaVA v1.5 [5], que ostenta 13 billones de parámetros y se fundamenta en LLaMA [6] como modelo de lenguaje base.

El proceso comienza con la extracción de las características visuales de la imagen mediante el encoder visual ViT-L/14, que transforma la imagen de entrada en embeddings vectoriales $Z_v$ según la siguiente expresión:

$$Z_v = g(X_v) \tag{1}$$

Estos embeddings visuales son luego mapeados al espacio vectorial del modelo de lenguaje grande (LLM) utilizando una matriz de transformación $W$ preentrenada, en este caso perteneciente a LLaMA, como se muestra a continuación:


$$H_v = W \cdot Z_v \tag{2}$$

Simultáneamente, el texto de entrada $X_q$ es convertido en un embedding del modelo de lenguaje LLaMA $H_q$. La concatenación de estos dos conjuntos de embeddings, constituye la entrada combinada al LLM, que luego produce la respuesta $X_a$, resultando en una descripción coherente y contextual de la caricatura.

En la **Figura 4** se presenta la arquitectura de red utilizada por LLaVA. Esta fue extraida del paper "Visual Instruction Tuning" escrito por H. Liu, C. Li, Q. Wu, and Y. J. Lee.

<figure>
<center><img src="Paper Images/LLaVA Architecture.png" style="width:60%"></center>
<figcaption align = "center"> Figura 4. Arquitectura de red LLaVA</figcaption>
</figure>

Se desarrolló un módulo en Python específicamente diseñado para simplificar la interacción con LLaVA v1.5, ofreciendo dos funciones de fácil manejo. Además, este módulo está equipado con la capacidad de evaluar el rendimiento de las descripciones generadas mediante la comparación con descripciones de referencia, utilizando el BERT Score como métrica de evaluación. 

A continuación, se utiliza dicho módulo para generar descripciones automáticas para todas las imágenes procesadas y almacenadas en el directorio cropped_processing_images, las cuales se guardan localmente. Esto permite una posterior comparación detallada y análisis de rendimiento.

In [11]:
MMLM_origin_folder = "cropped_processed_images"

MMLM = IPRG.ImagePromptResponseGenerator(model_path="liuhaotian/llava-v1.5-13b",
                                        device="cuda:0",
                                        temperature=0,
                                        max_new_tokens=1024,
                                        load_4bit=True)

MMLM.generate_captions(MMLM_origin_folder)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Image cropped_processed_images/17.jpg loaded succesfully!
Image cropped_processed_images/1.jpg loaded succesfully!
Image cropped_processed_images/3.jpg loaded succesfully!
Image cropped_processed_images/18.jpg loaded succesfully!
Image cropped_processed_images/15.jpg loaded succesfully!
Image cropped_processed_images/11.jpg loaded succesfully!
Image cropped_processed_images/5.jpg loaded succesfully!
Image cropped_processed_images/2.jpg loaded succesfully!
Image cropped_processed_images/6.jpg loaded succesfully!
Image cropped_processed_images/9.jpg loaded succesfully!
Image cropped_processed_images/13.jpg loaded succesfully!
Image cropped_processed_images/7.jpg loaded succesfully!
Image cropped_processed_images/8.jpg loaded succesfully!
Image cropped_processed_images/4.jpg loaded succesfully!
Image cropped_processed_images/10.jpg loaded succesfully!
Image cropped_processed_images/14.jpg loaded succesfully!
Image cropped_processed_images/19.jpg loaded succesfully!
Image cropped_processed

## Resultados

En esta sección se presentan los resultados obtenidos de la aplicación de los métodos descritos previamente, tanto en aspectos cualitativos como cuantitativos. Los resultados cuantitativos se basan en el promedio de las métricas aplicadas a un conjunto de 100 imágenes proporcionadas por el cliente. Para la extracción de texto y objetos, se utilizó el índice de Jaccard, mientras que la calidad de las descripciones generadas se midió con la puntuación de BERT, comparándolas con las anotaciones de referencia realizadas por un experto. Los detalles de estas métricas se exploran en la sección subsiguiente.

### Métricas Utilizadas

- **Índice de Jaccard**

El índice de Jaccard es una medida estadística utilizada para evaluar la similitud y diversidad entre conjuntos de muestras. Se define como el tamaño de la intersección dividido por el tamaño de la unión de los conjuntos de muestras. Matemáticamente, se puede expresar como:

$$\frac{|A \cap B|}{|A \cup B|}$$

El índice de Jaccard es un número entre 0 y 1. Un valor de 0 indica que los conjuntos no tienen elementos en común, mientras que un valor de 1 indica que todos los elementos de los conjuntos son comunes, es decir, los conjuntos son idénticos.

* **Puntuación de BERT**

El BERT Score es una medida que evalúa la calidad de las traducciones de texto basándose en la representación semántica de los embeddings de BERT (Bidirectional Encoder Representations from Transformers).

BERT Score presenta 3 metricas para la comparación, Precision, Recall y F1-Score:


 - **Precision**: para cada token en texto generado, se busca el token más similar (similitud del coseno) en texto de referencia (ground true).

 - **Recall**:  para cada token en texto referencia (ground true), se busca el token más similar (similitud del coseno) en texto generado.

La formula representativa de esto, para el caso de Recall, sería:
$$ R_{\text{BERT}} = \frac{1}{|x|} \sum_{x_i \in x} \max_{x_j \in \hat{x}} x_i^\top \hat{x}_j $$


En este caso de interés, donde se desea comparar el caption generado por una descripción automática contra la descripción etiquetada por un experto, se elige la medida de F1 del BERT Score como métrica para la comparación. Esto es así porque la métrica F1 proporciona un equilibrio entre precisión y exhaustividad, siendo particularmente efectiva en situaciones donde se busca evaluar la similitud semántica entre dos textos. Al utilizar el BERT Score con la métrica F1, se penaliza tanto la inclusión de tokens irrelevantes en nuestro caption generado (lo que disminuiría la precisión) como la omisión de tokens importantes presentes en la descripción real (afectando la exhaustividad). De esta manera, se obtiene una medida más equilibrada y representativa de la calidad de la coincidencia entre el caption generado y el etiquetado por el experto.

### Resultados Preprocesamiento

- **Algoritmo de recorte Inteligente**

El algoritmo de recorte inteligente se implementó en un conjunto de 100 imágenes de muestra. Posteriormente, el experto revisó detalladamente los resultados para asegurar la calidad del recorte. Esta revisión se centró en verificar dos aspectos clave: primero, que no quedaran espacios en blanco innecesarios en las imágenes, y segundo, que el recorte no eliminara más contenido del necesario. Tras una evaluación exhaustiva, el experto confirmó que el 100% de las imágenes fueron recortadas con precisión impecable.

A continuación, en la **Figura 5** presentamos una muestra representativa de 5 imágenes, mostrando el antes y después del proceso de recorte, para ilustrar la efectividad del algoritmo.

In [ ]:
display_images_in_rows(["original_images/1.jpg", "original_images/2.jpg", "original_images/101.jpg",  "original_images/69.jpg",  "original_images/84.jpg"], ["cropped_images/1.jpg", "cropped_images/2.jpg", "cropped_images/101.jpg",  "cropped_images/69.jpg",  "cropped_images/84.jpg"], "Imagen Original", "Imagen Recortada", save_path="Paper Images/Resultados Recorte.png")

<figure>
<center><img src="Paper Images/Resultados Recorte.png" style="width:60%"></center>
<figcaption align = "center"> Figura 5. Resultados Recorte Inteligente - Imágenes Originales vs Imágenes Recortadas</figcaption>
</figure>

- **Mejora de la Calidad Visual Mediante Técnicas Tradicionales de Procesamiento de Imágenes**

Al igual que el algoritmo de recorte inteligente, el algoritmo de mejora de calidad visual mediante técnicas tradicionales se implementó sobre el conjunto de 100 imágenes de muestra. Posteriormente, se evaluó de manera cualitativa (visualmente) y se observó una clara mejoría en la cálidad de la imagen, asímismo, se compararon las métricas cuantitativas obtenidas en los módulos posteriores y se observó cómo el preprocesamiento aplicado mejoraba dichas métricas .

La **Figura 6** muestra la imagen resultante de cada una de las etapas del algoritmo descrito en la sección "Método". Por su parte, la **Figura 7** presenta una comparación de 5 imágenes antes y después del proceso de mejora, ilustrando de manera efectiva la capacidad del algoritmo para optimizar la calidad visual.

In [ ]:

image, gray_image, blurred_image, clahe_image, adaptive_binary_image = preprocess_image("cropped_images/71.jpg", True)
fig, axes = plt.subplots(1, 5, figsize=(30, 15))
ax = axes.ravel()
ax[0].imshow(image)
ax[0].set_title("Original")
ax[1].imshow(gray_image, cmap='gray')
ax[1].set_title("Escala de grises")
ax[2].imshow(blurred_image, cmap='gray')
ax[2].set_title("Filtro Gaussiano")
ax[3].imshow(clahe_image, cmap='gray')
ax[3].set_title("CLAHE")
ax[4].imshow(adaptive_binary_image, cmap='gray')
ax[4].set_title("Umbral Adaptativo")
for a in ax:
    a.axis('off')
plt.tight_layout()
plt.savefig("Paper Images/Etapas Preprocesamiento Tradicional.png", bbox_inches='tight')
plt.close()

<figure>
<center><img src="Paper Images/Etapas Preprocesamiento Tradicional.png" style="width:80%"></center>
<figcaption align = "center"> Figura 6. Imágen resultante por cada etapa del algoritmo implementado</figcaption>
</figure>

In [ ]:
display_images_in_rows(["cropped_images/4.jpg", "cropped_images/8.jpg",  "cropped_images/12.jpg",  "cropped_images/48.jpg",  "cropped_images/90.jpg"], ["cropped_processed_images/4.jpg", "cropped_processed_images/8.jpg", "cropped_processed_images/12.jpg", "cropped_processed_images/48.jpg",  "cropped_processed_images/90.jpg"], "Imagen Recortada", "Imagen Preprocesada - Téncicas Tradicionales", save_path="Paper Images/Resultados Preprocesamiento Tradicional.png", gray=True)

<figure>
<center><img src="Paper Images/Resultados Preprocesamiento Tradicional.png" style="width:80%"></center>
<figcaption align = "center"> Figura 6. Comparación Imagen antes y después de ser preprocesada con Técnicas Tradicionales</figcaption>
</figure>

### Resultados Extracción de Texto

Con el fin de evaluar el desempeño en la extracción de textos se encontró el índice de Jaccard Promedio sobre las 100 imágenes proporcionadas por la experta. Este se calculó tal y como se explicó en la subsección de métricas. En este caso uno de los conjuntos es la la lista de palabras predecidas y el otro es la lista de palabras reales (anotadas por la experta).

En la **Figura 7** y **Figura 8** se muestran 2 ejemplos de la salida del módulo de extracción de textos.

<figure>
<center><img src="Paper Images/OCR_1.png" style="width:80%"></center>
<figcaption align = "center"> Figura 7. Ejemplo de salida extracción de Texto</figcaption>
</figure>

<figure>
<center><img src="Paper Images/OCR_2.png" style="width:80%"></center>
<figcaption align = "center"> Figura 8. Ejemplo de salida extracción de Texto</figcaption>
</figure>

A continuación, se cargan las etiquetas reales tanto de texto como de objetos que se encuentran en el archivo 'labels_and_ocrtext.csv', este archivo fue entregado por nuestra experta.

In [3]:
df_true_text, df_true_objects = prepare_labels(pd.read_csv("labels_and_ocrtext.csv"))
OCR.set_df_true_text(df_true_text)

En seguida, se cálcula el índice de Jaccard para cada imagen, además, se calcula el índice de Jaccard promedio sobre las 100 imágenes. **Se puede ver que se obtuvo un índice de jaccard promedio de 0.403.**

In [4]:
OCR.get_df_results()
print(f"El Índice de Jaccard Promedio de las 100 Imágenes es: {OCR.calculate_jaccard_index()[1]}")
OCR.calculate_jaccard_index()[0]

El Índice de Jaccard Promedio de las 100 Imágenes es: 0.4036019802547748


,id,texts,true_text,jaccard_index
0,88,"[tote, patdel, fiasco, de, la, comision, velis...","[de, comision, grabado, que, reff, patada, fia...",0.352941
1,84,"[surtido, permanente, de, papel, de, coloadura...","[variedad, clases, paisajes, surtido, papel, 1...",0.189189
2,27,"[vida, uln, es, asicla, actos, cuatro, comedia...","[vida, escena, o, muda, actual, iii, las, come...",0.523810
3,20,"[ser, den, porie, metenerezl, actual, prefecto...","[melendez, de, provincia, don, colon, sr, actu...",0.333333
4,26,"[wimueran, los, nombress, uomueran, s]","[los, mueran, motas, hombres]",0.125000
...,...,...,...,...
96,38,"[ooo, aont, unto, rorcolo, comicod, madrid, el...","[cierto, que, secreto, pueden, pongo, hacer, j...",0.684211
97,12,"[nicaragua, amtolio, tratadd, he, ran, hay, a2...","[de, hay, herran, nicaragua, capitolio, entier...",0.437500
98,56,"[las, mujeres, en, el, mercado, caxicatura, po...","[mercado, grabado, el, en, reff, caricatura, m...",0.400000
99,16,"[stots, co, micas, 48]","[comicas, siluetas]",0.000000


### Resultados Generación de Descripciones

Para evaluar la cálidad de las descripciones generadas por el LMM se cálculo el F1 Promedio de la puntuación de BERT sobre las 100 imágenes. Este se calculó tal y como se explicó en la subsección de métricas. En este caso  se hace la comparación del caption generado con el caption proporcionado por la experta.

A continuación, se ejemplifica el uso del módulo de LMM para la generación de un caption de una de las caricaturas.

<figure>
<center><img src="Paper Images/4.jpg" style="width:30%"></center>
<figcaption align = "center"> Figura 9. Imagen utilizada como ejemplo para generar el caption.</figcaption>
</figure>

In [7]:
MMLM.load_image("cropped_processed_images/4.jpg")
print("\nDescripción generado por el modelo: ")
print("\n"+MMLM.generate_response("Describe en español esta imagen"))

Image cropped_processed_images/4.jpg loaded succesfully!

Descripción generado por el modelo: 

La imagen es una ilustración antigua en blanco y negro que muestra a dos hombres y un perro. Uno de los hombres está tocando el trombón, mientras que el otro está tocando la flauta. El perro está en el medio, entre los dos hombres, y parece estar atento a la música.

La imagen tiene un estilo de dibujo que evoca la sensación de un cuadro antiguo. Los personajes y el perro están bien definidos, y la composición de la imagen es equilibrada, con los tres elementos en una línea horizontal.</s>


A continuación se muestran las primeras 10 descripciones generadas:

In [14]:
MMLM.get_df_results().head(10)

,image,texts
0,cropped_processed_images/17,La imagen es una fotografía antigua de un homb...
1,cropped_processed_images/1,La imagen es una fotografía en blanco y negro ...
2,cropped_processed_images/3,La imagen es una ilustración en blanco y negro...
3,cropped_processed_images/18,La imagen es una impresión antigua de un escud...
4,cropped_processed_images/15,La imagen es una ilustración en blanco y negro...
5,cropped_processed_images/11,La imagen es una página de un periódico en bla...
6,cropped_processed_images/5,La imagen es una serie de cuatro cuadros en bl...
7,cropped_processed_images/2,La imagen es una ilustración antigua en blanco...
8,cropped_processed_images/6,La imagen es una ilustración en blanco y negro...
9,cropped_processed_images/9,La imagen es una ilustración antigua en blanco...


Por último, se calcula el BERT score promedio por cada una de las 3 métricas disponibles (Precision, Recall y F1) sobre las 100 imágenes disponibles. **Se puede observar que se obtuvó una precisión promedio de 0.28, un recall promedio de 0.14 y un F1 promedio de 0.21.**

In [12]:
MMLM.get_bert_score()

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.32 seconds, 60.11 sentences/sec


{'precision': 0.2885325849056244,
 'recall': 0.13698138296604156,
 'f1': 0.21049249172210693}

# Conclusiones

Se ha logrado un avance significativo en la mejora de la calidad visual y la legibilidad de las caricaturas del siglo XIX. Los procedimientos de preprocesamiento y técnicas tradicionales de procesamiento de imágenes han demostrado ser efectivos en optimizar la nitidez y la claridad, como se evidencia en las comparativas visuales y mejoras cuantitativas de métricas en módulos subsecuentes.

 La implementación del modelo de mejora de imagen REAL-ESGRAN ha demostrado ser particularmente beneficiosa en la reescalización y en la eliminación de ruido de las imágenes. Esto se traduce en un aumento de la resolución que mejora significativamente la definición y la legibilidad de los textos y detalles en las caricaturas, como se ha evidenciado en las imágenes de muestra que representan diversas categorías y condiciones del conjunto de datos. Lo anterior es sumamente beneficioso para el proceso de restauración de las imágenes en general.

Por su parte, el flujo de trabajo para la identificación automática de texto, el etiquetado y la generación de descripciones ha facilitado significativamente el análisis e interpretación de fuentes visuales históricas. Este avance representa un hito en la digitalización y preservación del arte caricaturesco del siglo XIX, permitiendo un acceso más amplio y una comprensión más profunda de estos recursos culturales.

 A pesar de los avances, aún hay margen para mejorar las métricas de rendimiento en cada uno de los módulos implementados. Una ruta prometedora para la optimización es el fine-tuning de los modelos con un mayor volumen de datos sintéticos, lo cual podría conducir a una mejora en la precisión y generalización de los algoritmos aplicados.

 Finalmente, se concluye que la tarea de identificar y entender las características textuales en las imágenes del siglo XIX continúa siendo un desafío considerable. Este trabajo es esencial no solo para apreciar la estética y el estilo visual de la época, sino también para comprender la evolución del lenguaje visual y textual a lo largo del tiempo.

[1] https://github.com/faustomorales/keras-ocr

[2] https://github.com/clovaai/CRAFT-pytorch

[3] https://github.com/janzd/CRNN

[4] https://github.com/shegocodes/keras-ocr

[5] https://github.com/haotian-liu/LLaVA

[6] https://github.com/facebookresearch/llama